<a href="https://colab.research.google.com/github/ismailukman/BinaryTree/blob/master/baseline_model_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DSS images

Here we'll be training a CNN from the scratch to classify the DSS images. We'll start, as usual, by importing our usual resources. And checking if we can train our model on GPU. The hyper-parameters used were obtained by training on very low resolution images, but there is a lot of scope for additional tuning.

### Ensure data available

If the data is not present in a specified sub-directory download from my ebi home directory
Download the DSS data from [this link](https://www.ebi.ac.uk/~kola/low_res_images_epithelium_224x224_test_train_valid.tar.gz), save it in the home directory of this notebook and extract the tar.gz file to get the directory `low_res_images_epithelium_224x224/`. **Change the data_dir variable below to reflect this directory**.

In [0]:
import os
import numpy as np
import torch

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [0]:
!wget https://www.ebi.ac.uk/~kola/low_res_images_epithelium_224x224_test_train_valid.tar.gz

In [0]:
!tar -zxvf low_res_images_epithelium_224x224_test_train_valid.tar.gz

## Load and Transform our Data

We'll be using PyTorch's [ImageFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder) class which makes is very easy to load data from a directory. For example, the training images are all stored in a directory path that looks like this:
```
root/class_1/xxx.png
root/class_1/xxy.png
root/class_1/xxz.png

root/class_2/123.png
root/class_2/nsdf3.png
root/class_2/asd932_.png
```

Where, in this case, the root folder for training is `low_res_images_epithelium_224x224/train/` and the classes are the scores given to the images.

In [0]:
# define training and test data directories
# THIS DIRECTORY IS WHERE YOU EXTRACTED THE IMAGES TO
data_dir = 'low_res_images_epithelium_224x224/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

# classes are folders in each directory with these names
classes = ['0.0', '0.5', '1.0', '1.5', '2.0', '3.0']

Transforming the Data
Here we are using the same set of images used for transfer learning with the VGG16 pre-trained model. When we perform transfer learning, we have to shape our input data into the shape that the pre-trained model expects. VGG16 expects 224-dim square images as input and so, we resize each image to fit this mold.

In [0]:
# load and transform data using ImageFolder

# VGG-16 Takes 224x224 images as input, so we resize all of them (we will use VGG16 later)
data_transform = transforms.Compose([transforms.RandomResizedCrop(224), 
                                      transforms.ToTensor()])

train_data = datasets.ImageFolder(train_dir, transform=data_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

# print out some data stats
n_train = len(train_data)
n_valid = len(valid_data)
n_test = len(test_data)
print('Num training images: ', n_train)
print('Num validation images: ', n_valid)
print('Num test images: ', n_test)

In [0]:
# define dataloader parameters
batch_size = 20
num_workers=0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [0]:
# Visualize some sample data

# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])

---
## Define the Model

To define a model for training we use hyperparameters obtained from training very low resolution images:<br>
    * A few convolutional layers
    * One fully connected layer
    * One softmax (output) layer

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        nn.init.kaiming_normal_(self.conv1.weight)
        self.conv2 = nn.Conv2d(32, 64, 3)
        nn.init.kaiming_normal_(self.conv2.weight)
        self.conv3 = nn.Conv2d(64, 128, 3)
        nn.init.kaiming_normal_(self.conv3.weight)
        
        self.drop2d = nn.Dropout2d(0.25)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 26 * 26, 200)
        self.fc2 = nn.Linear(200, 6)
        self.drop1d = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.drop2d(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drop2d(x)
        x = self.pool(F.relu(self.conv3(x)))
        
        # Each unpadded 2d convolution reduces dimensions by 2, each maxpool by factor of 0.2
        x = x.view(-1, 128 * 26 * 26) 
        x = F.relu(self.fc1(x))
        x = self.drop1d(x)
        # When using keras I applied softmax to output. Not sure what difference this makes
        #x = F.softmax(self.fc2(x),dim=0)
        x = self.fc2(x)
        
        return x

model = Net()
print(model)

# If GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()

In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = torch.nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Use scheduler to decrease learning rate every step_size epochs
step_size = 10
scheduler = optim.lr_scheduler.StepLR(optimizer, 20, gamma=0.5, last_epoch=-1)

In [0]:
# number of epochs to train the model
n_epochs = 50

## TODO complete epoch and training batch loops
## These loops should update the classifier-weights of this model
## And track (and print out) the training loss over time

for epoch in np.arange(1,n_epochs+1):
    training_loss = 0.0
    model.train()
    scheduler.step()
    for i, inputs in enumerate(train_loader):
        data, target = inputs
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()
        
    #Apply model to validation set
    model.eval()
    validation_loss = 0.0
    for i, inputs in enumerate(valid_loader):
        data, target = inputs
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        outputs = model(data)
        loss = criterion(outputs, target)
        validation_loss += loss.item()    
        
    print('{0:d}, training_loss: {1:.5f}, validation_loss: {2:.5f}'.format(epoch, training_loss/n_train, validation_loss/n_valid))
print('Finished Training')


---
## Testing

Below you see the test accuracy for each class.

In [0]:
# Function to create confusion matrix
import pandas as pd
import numpy as np
def conf_mat(row_arr,col_arr):
    if not np.all(row_arr.shape == col_arr.shape):
        print("Need shapes of both arrays to be equal")
        return False
    n = row_arr.shape[-1]
    unique_values = list(set(row_arr[:]).union(set(col_arr[:])))
    unique_values.sort()
    n2 = len(unique_values)
    cmat = np.zeros((n2,n2),np.int)
    
    for r, c in zip(row_arr, col_arr):
        row_index = unique_values.index(r)
        col_index = unique_values.index(c)
        cmat[row_index,col_index] += 1

    return pd.DataFrame(data = cmat, index=unique_values, columns=unique_values)

In [0]:
# track test loss over all 6 classes
n_classes = 6
test_loss = 0.0
class_correct = list(0. for i in range(n_classes))
class_total = list(0. for i in range(n_classes))

# Also create a confusion matrix to give insight into results
list_targets = []
list_outputs = []

# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    # test_loss += loss.item()*data.size(0) - correct way to calculate loss, but use same metric as for training and validation
    test_loss += loss.item()
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(target.data.size()[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
    
    # Add to list for calculating confusion matrix
    list_outputs.extend(np.squeeze(pred.numpy()) if not train_on_gpu else np.squeeze(pred.cpu().numpy()))
    list_targets.extend(np.squeeze(target.numpy()) if not train_on_gpu else np.squeeze(target.cpu().numpy()))

# calculate avg test loss
test_loss = test_loss/n_test
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(n_classes):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

print('\nConfusion matrix:')
print(conf_mat(np.array(list_targets),np.array(list_outputs)))

In [0]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title("{} ({})".format(classes[preds[idx]], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))